<a href="https://colab.research.google.com/github/aryankargwal/Grade-Predictor_Linear_Regression/blob/master/Grade_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
import unittest
import torch
from sklearn.model_selection import train_test_split, cross_val_score,KFold
from sklearn import linear_model
from math import sqrt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!wget https://raw.githubusercontent.com/aryankargwal/Grade-Predictor_Linear_Regression/master/Datasets/train.csv

--2020-06-26 10:26:39--  https://raw.githubusercontent.com/aryankargwal/Grade-Predictor_Linear_Regression/master/Datasets/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38031 (37K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  37.14K  --.-KB/s    in 0.01s   

2020-06-26 10:26:39 (3.17 MB/s) - ‘train.csv’ saved [38031/38031]



In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df

,Unnamed: 0,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,39,GP,F,15,R,GT3,T,2,2,at_home,other,reputation,mother,1,1,0,yes,yes,yes,yes,yes,yes,no,no,4,3,1,1,1,2,8,14,13,13
1,56,GP,F,15,U,GT3,A,4,3,services,services,reputation,mother,1,2,0,no,yes,yes,yes,yes,yes,yes,no,4,3,2,1,1,1,0,14,15,15
2,30,GP,M,15,U,GT3,T,4,4,health,services,home,mother,1,2,0,no,yes,yes,no,no,yes,yes,no,5,4,2,3,4,5,0,9,11,12
3,377,MS,F,18,R,LE3,T,4,4,teacher,services,course,mother,1,2,0,no,no,yes,yes,yes,yes,yes,no,5,4,3,3,4,2,4,8,9,10
4,386,MS,F,18,R,GT3,T,4,4,teacher,at_home,reputation,mother,3,1,0,no,yes,yes,yes,yes,yes,yes,yes,4,4,3,2,2,5,7,6,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,71,GP,M,15,U,GT3,T,4,2,other,other,course,mother,1,4,0,no,no,no,no,yes,yes,yes,no,3,3,3,1,1,3,0,10,10,10
341,106,GP,F,15,U,GT3,T,2,2,other,other,course,mother,1,4,0,yes,yes,yes,no,yes,yes,yes,no,5,1,2,1,1,3,8,7,8,8
342,270,GP,F,19,U,GT3,T,3,3,other,services,home,other,1,2,2,no,yes,yes,yes,yes,yes,yes,no,4,3,5,3,3,5,15,9,9,9
343,348,GP,F,17,U,GT3,T,4,3,health,other,reputation,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,4,4,3,1,3,4,0,13,15,15


In [ ]:
#popping the last column 
final_grade = df.pop('G3')

In [ ]:
#renaming and popping the id column
df.rename( columns={'Unnamed: 0':'id'}, inplace=True )
ID=df.pop('id')

# Preparing Columns according to their data type

First of due to lack of much of a numerical or rather data which can have any numerical value we will straight get to the categorical columns of the dataset.

Starting with the categorical data lets just put all the columns in categorical_columns.

In [ ]:
categorical_columns = [col for col in df.columns]

In [ ]:
for col_name, col in df[categorical_columns].items():
    df[col_name] = col.astype('category').cat.as_ordered()

Lets now define ordinal columns which are the categorical columns which can have a certain ordered set of values, ie. each value have a certain unit distance between them.

In [ ]:
ordinal_column_data=[
     ('famsize',['LE3','GT3']),
     ('Medu',list(range(0,6))),
     ('Fedu',list(range(0,6))),
     ('traveltime',list(range(0,5))),
     ('studytime',list(range(0,5))),
     ('famrel',list(range(0,6))),
     ('freetime',list(range(0,6))),
     ('goout',list(range(0,6))),
     ('Dalc',list(range(0,6))),
     ('Walc',list(range(0,6))),
     ('health',list(range(0,6))),
     ('age',list(range(15,22))),
     ('failures',list(range(0,5))),
     ('absences',list(range(0,94))),
     ('G1',list(range(0,20))),
     ('G2',list(range(0,20)))
]
ordinal_columns = [o[0] for o in ordinal_column_data]

for col, categories in ordinal_column_data:
    df[col].cat.set_categories(categories, ordered=True, inplace=True)

In [ ]:
ordinal_column_data

[('famsize', ['LE3', 'GT3']),
 ('Medu', [0, 1, 2, 3, 4, 5]),
 ('Fedu', [0, 1, 2, 3, 4, 5]),
 ('traveltime', [0, 1, 2, 3, 4]),
 ('studytime', [0, 1, 2, 3, 4]),
 ('famrel', [0, 1, 2, 3, 4, 5]),
 ('freetime', [0, 1, 2, 3, 4, 5]),
 ('goout', [0, 1, 2, 3, 4, 5]),
 ('Dalc', [0, 1, 2, 3, 4, 5]),
 ('Walc', [0, 1, 2, 3, 4, 5]),
 ('health', [0, 1, 2, 3, 4, 5]),
 ('age', [15, 16, 17, 18, 19, 20, 21]),
 ('failures', [0, 1, 2, 3, 4]),
 ('absences',
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   

As for the left columns in categorical_columns they can be defined as nominal columns, which means the values in them dont really have any distance between them. For example. Male or Female there is no exact heirarchy between them so they can treated as nominal columns.

In [ ]:
# defining other categorical columns without any ordinal relation
other_cat_columns = [col for col in categorical_columns if col not in ordinal_columns]

In this next step we basically provide a numerical value to each of the categorical values and a seperate step for basically making dummy columns for the nominal data and add them to the numerical version of the dataset.

What the dummy aspect is doing here is basically adding seperate columns for all the unordered categorical values and treating them as a seperate feature.

In [ ]:
df_numeric = df.copy()
dummies = pd.get_dummies(df_numeric[other_cat_columns], dummy_na=True)
for col_name in categorical_columns:
    # Use +1 to push the -1 NaN value to 0
    df_numeric[col_name] = df_numeric[col_name].cat.codes + 1

In [ ]:
df_numeric.drop(other_cat_columns, axis=1, inplace=True)
df_numeric = pd.concat([df_numeric, dummies], axis=1)

In [ ]:
df_numeric

,age,famsize,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,school_GP,school_MS,school_nan,sex_F,sex_M,sex_nan,address_R,address_U,address_nan,Pstatus_A,Pstatus_T,Pstatus_nan,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Mjob_nan,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,Fjob_nan,reason_course,reason_home,reason_other,reason_reputation,reason_nan,guardian_father,guardian_mother,guardian_other,guardian_nan,schoolsup_no,schoolsup_yes,schoolsup_nan,famsup_no,famsup_yes,famsup_nan,paid_no,paid_yes,paid_nan,activities_no,activities_yes,activities_nan,nursery_no,nursery_yes,nursery_nan,higher_no,higher_yes,higher_nan,internet_no,internet_yes,internet_nan,romantic_no,romantic_yes,romantic_nan
0,1,2,3,3,2,2,1,5,4,2,2,2,3,9,15,14,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0
1,1,2,5,4,2,3,1,5,4,3,2,2,2,1,15,16,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0
2,1,2,5,5,2,3,1,6,5,3,4,5,6,1,10,12,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0
3,4,1,5,5,2,3,1,6,5,4,4,5,3,5,9,10,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0
4,4,2,5,5,4,2,1,5,5,4,3,3,6,8,7,6,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,1,2,5,3,2,5,1,4,4,4,2,2,4,1,11,11,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0
341,1,2,3,3,2,5,1,6,2,3,2,2,4,9,8,9,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0
342,5,2,4,4,2,3,3,5,4,6,4,4,6,16,10,10,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0
343,3,2,5,4,2,4,1,5,5,4,2,4,5,1,14,16,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0


# Getting to the model training
Instead of seperating data into validation set, I am going to be using Kfold cross validation which basically translates to breaking data into n test/val sets train the model, evaluates result and averages them.

Also, I will be using Lasso model which is just Linear Regression with L1 regularization as normal Linear Regression may cause overfitting with such small data.

In [ ]:
kf = KFold(n_splits=10, shuffle=False, random_state= None)
model = linear_model.Lasso(alpha=0.0004)
scores = np.sqrt(
    -cross_val_score(model, df_numeric, final_grade, cv=kf, scoring='neg_mean_squared_error'))

In [ ]:
scores.mean()

1.9454846361639315